In [22]:
import csv
import re
import pathlib

epoch_pattern = re.compile(r"^starting epoch:\s*(\d+)")
starting_pattern = re.compile(r"^Starting training.*")
train_metrics_pattern = re.compile(r"Train -\s+loss:\s*([\d.]+)\s+top-1:\s*([\d.]+)\s+top-5:\s*([\d.]+)\s+top-10:\s*([\d.]+)")
eval_metrics_pattern = re.compile(r"Eval -\s+loss:\s*([\d.]+)\s+top-1:\s*([\d.]+)\s+top-5:\s*([\d.]+)\s+top-10:\s*([\d.]+)")

name_var_pattern = re.compile(r"-?((?P<varname>[a-zA-Z\-_]+)=(?P<value>\-?\d+\.?\d*(e\-?\d+)?))")
name_pattern = re.compile(r"^(?P<exp_group>[\d]+)?\-?(?P<Layer>.+)(?P<Position>Pre|Post|Both)(?P<DyTfn>.*tanh)?(?P<Vars>(([a-zA-Z\-_]+)=([\d\.e\-]+))*)$")


def parse_name(name):
    out = {"exp_group":1, 'lr':1e-4, 'dim_hidden':256}
    search = name_pattern.search(name)
    if search is None:
        out["Layer"] = "Identity"
        out["Position"] = "Both"
        return out
    out["Layer"] = search.group("Layer")
    if out["Layer"]=="DyT":
        out["Alpha"] = 0.5
        out["DyTfn"] = "Tanh"
    out["Position"] = search.group("Position")
    if search.group("DyTfn") is not None:
        out["DyTfn"] = search.group("DyTfn")
    if search.group("exp_group") is not None:
        out["exp_group"] = int(search.group("exp_group"))
    out.update({e[1]:float(e[2]) for e in name_var_pattern.findall(search.group("Vars"))})
    return out

def read_log(folder):
    data_rows = []
    current_epoch = None
    train_metrics = {}
    eval_metrics = {}
    english = None
    french = None
    current_model = folder.name
    smallest_epoch = None
    
    with open(folder/"training_logs.log", "r", encoding="utf-8") as f:
        current_model_data = []
        for line in f.readlines():
            line = line.strip()
            
            if match := starting_pattern.match(line):
                continue

            if match := epoch_pattern.match(line):
                if smallest_epoch is None:
                    smallest_epoch = int(match.group(1))
                current_epoch = int(match.group(1))-smallest_epoch
                continue

            if match := train_metrics_pattern.match(line):
                train_metrics = {
                    "Train Loss": float(match.group(1)),
                    "Train Top-1": float(match.group(2)),
                    "Train Top-5": float(match.group(3)),
                    "Train Top-10": float(match.group(4)),
                }
                continue

            if match := eval_metrics_pattern.match(line):
                eval_metrics = {
                    "Eval Loss": float(match.group(1)),
                    "Eval Top-1": float(match.group(2)),
                    "Eval Top-5": float(match.group(3)),
                    "Eval Top-10": float(match.group(4)),
                }
                continue

            if english is None:
                english = line
                continue
            elif french is None:
                french = line
                row = {
                    **parse_name(current_model),
                    "Exp_full_name": current_model,
                    "Epoch": current_epoch,
                    "Ground Truth": english,
                    "Prediction": french,
                    **train_metrics,
                    **eval_metrics,
                }
                current_model_data.append(row)
                english = None
                french = None
                continue

    data_rows.extend(current_model_data)
    current_model_data = []
    return data_rows



In [23]:
runs = [e for e in pathlib.Path("logs").glob("*") if e.is_dir()]

In [24]:
logs = []
for e in runs: 
    logs += read_log(e)

In [25]:
data = {k:[] for e in logs for k in e.keys()}

for e in logs:
    for k in data.keys():
        data[k].append(e.get(k))


In [26]:
import pandas as pd

In [27]:
cols = ["Layer","Position","Epoch","Train Loss","Eval Loss","Alpha","DyTfn","exp_group","lr","dim_hidden","Exp_full_name","Ground Truth","Prediction","Train Top-1","Train Top-5","Train Top-10","Eval Top-1","Eval Top-5","Eval Top-10"]

df = pd.DataFrame(data, columns=cols)
df.to_csv("out.csv", index=False)

In [28]:
df.describe()

,Epoch,Train Loss,Eval Loss,Alpha,exp_group,lr,dim_hidden,Train Top-1,Train Top-5,Train Top-10,Eval Top-1,Eval Top-5,Eval Top-10
count,210.000000,210.000000,210.000000,140.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000,210.000000
mean,2.000000,3.458476,3.850857,0.355357,1.333333,0.000113,274.285714,0.418476,0.606333,0.665667,0.407714,0.579190,0.634667
std,1.417593,1.457839,2.437972,0.210916,0.472531,0.000071,66.087620,0.196838,0.198801,0.184211,0.216201,0.238732,0.230051
min,0.000000,1.240000,1.300000,0.050000,1.000000,0.000025,256.000000,0.090000,0.260000,0.330000,0.030000,0.070000,0.090000
25%,1.000000,2.140000,2.052500,0.050000,1.000000,0.000100,256.000000,0.220000,0.430000,0.510000,0.170000,0.320000,0.410000
50%,2.000000,3.130000,3.035000,0.500000,1.000000,0.000100,256.000000,0.480000,0.660000,0.710000,0.490000,0.670000,0.720000
75%,3.000000,4.822500,5.500000,0.500000,2.000000,0.000100,256.000000,0.590000,0.780000,0.830000,0.600000,0.790000,0.837500
max,4.000000,5.890000,14.880000,0.500000,2.000000,0.000400,512.000000,0.720000,0.900000,0.930000,0.710000,0.890000,0.920000


In [29]:
import numpy as np

In [31]:
df.query("Epoch==4 & exp_group==1").groupby(["Layer", "Position"]).agg({"Eval Loss": np.min})

/tmp/ipykernel_49045/2145597980.py:1: FutureWarning: The provided callable <function min at 0x7ffaef2f5580> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  df.query("Epoch==4 & exp_group==1").groupby(["Layer", "Position"]).agg({"Eval Loss": np.min})


Eval Loss
Layer     Position           
BatchNorm Both           1.73
          Post           1.74
          Pre            1.69
DyT       Both           3.23
          Post           3.25
          Pre            1.78
Identity  Both           1.71
Layernorm Both           1.77
          Post           1.77
          Pre            1.70
RMSNorm   Both           1.77
          Post           1.78
          Pre            1.70